# GANCA Results Visualization


## Snapshot to Voxel

Code used for turning snapshots in the generation into voxel plots

In [2]:
# import stuff

import data_helper
from data_helper import GANCA3DDataModule
from models import VoxelPerceptionNet, VoxelUpdateNet, VoxelNCAModel, VoxelDiscriminator
import visualise_helper
import utils
%load_ext autoreload
from tqdm.notebook import tqdm
%autoreload 2

import numpy as np
import sys
import torch
import random
import matplotlib.pyplot as plt
from pytorch_lightning import Trainer
import pytorch_lightning as pl
import math

from loguru import logger as gurulogger
gurulogger.remove()
gurulogger.add(sys.stdout, colorize=True, format="<blue>{time}</blue> <level>{message}</level>")
gurulogger.level("INFO", color="<red><bold>")

import os
from einops import rearrange
import torch.nn.functional as F
import pandas as pd

BLOCK2VEC_OUT_PATH = 'output/block2vec saves/block2vec 64 dim locked air/'
NUM_WORKERS = int(os.cpu_count() / 2)

# disable plt showing plots in notebook
plt.ioff()
%matplotlib agg
%matplotlib agg

In [3]:
embedding, mcid2block, block2embeddingidx, embeddingidx2block, block2mcid = utils.get_embedding_info(BLOCK2VEC_OUT_PATH)
converter = utils.DataConverter(embedding, mcid2block, block2embeddingidx, embeddingidx2block, block2mcid)

In [4]:
def visualize_snapshots_in_path(
    max_epoch = 16, 
    snapshots_folder_path = 'lightning_logs/GANCA/version_15/gen_snapshots', 
    n_cols = 4,
    explode_world=False,
    trim_world=False,
    no_air = True,
    alpha = True,
    explode_alpha = True,
    trim_alpha = True,
    metric='euclidean',
):

    # Logged progress per epoch to image
    for i in tqdm(range(max_epoch), desc = 'world states loop'):
        loaded = np.load(snapshots_folder_path + f'/epoch_{i}.npy')
        fig = visualise_helper.states_to_graphs(
                loaded, 
                embedding.weight.cpu(), 
                n_cols=n_cols, 
                n_rows= int(loaded.shape[0] / n_cols), 
                converter_class=converter, 
                size_multiplier=4,
                explode=explode_world,
                trim=trim_world,
                no_air = no_air,
                metric = metric,
            )
        fig.suptitle(f'visualizing epoch {i}')
        fig.savefig(snapshots_folder_path + f'/epoch_{i}.png')
    
    if alpha:
        # Visualising alpha channel
        for i in tqdm(range(max_epoch), desc = 'alpha states loop'):
            loaded = np.load(snapshots_folder_path + f'/epoch_{i}_alpha.npy')
            fig = visualise_helper.alpha_states_to_graphs(
                    loaded, 
                    n_cols=n_cols, 
                    n_rows= int(loaded.shape[0] / n_cols), 
                    size_multiplier=4,
                    explode = explode_alpha, # this makes covered voxels visible
                    trim = trim_alpha, # remove uselsss white info
                )
            fig.suptitle(f'visualizing epoch {i}')
            fig.savefig(snapshots_folder_path + f'/epoch_{i}_alpha.png')

Possible paths:

- GANCA_MMGAN
- GANCA_WGAN
- GANCA_WGANGP
- GANCA_WGANGP_DUAL_D
- Deconv_WGANGP

In [6]:
visualize_snapshots_in_path(
    max_epoch = 16, 
    snapshots_folder_path = 'lightning_logs/GANCA_MMGAN/version_0/gen_snapshots', 
    n_cols = 4,
    explode_world=False,
    trim_world=False,
    no_air = True,
    alpha = True,
    explode_alpha = True,
    trim_alpha = True,
    metric='cosine',
)

world states loop:   0%|          | 0/16 [00:00<?, ?it/s]

n cols 4, n rows 24, input dims (96, 64, 32, 32, 32)


3D plots:   0%|                                                                                               …

TypeError: object of type 'NoneType' has no len()

In [ ]:
visualize_snapshots_in_path(
    max_epoch = 16, 
    snapshots_folder_path = 'lightning_logs/GANCA_WGAN/version_0/gen_snapshots', 
    n_cols = 4,
    explode_world=False,
    trim_world=False,
    no_air = True,
    alpha = True,
    explode_alpha = True,
    trim_alpha = True,
    metric='cosine',
)

In [ ]:
visualize_snapshots_in_path(
    max_epoch = 16, 
    snapshots_folder_path = 'lightning_logs/GANCA_WGANGP/version_0/gen_snapshots', 
    n_cols = 4,
    explode_world=False,
    trim_world=False,
    no_air = True,
    alpha = True,
    explode_alpha = True,
    trim_alpha = True,
    metric='cosine',
)

In [ ]:
visualize_snapshots_in_path(
    max_epoch = 16, 
    snapshots_folder_path = 'lightning_logs/GANCA_WGANGP_DUAL_D/version_0/gen_snapshots', 
    n_cols = 4,
    explode_world=False,
    trim_world=False,
    no_air = True,
    alpha = True,
    explode_alpha = True,
    trim_alpha = True,
    metric='cosine',
)

In [20]:
# visualize_snapshots_in_path(
#     max_epoch = 64, 
#     snapshots_folder_path = '/root/autodl-tmp/lightning_logs/Deconv_WGANGP/version_1/gen_snapshots', 
#     n_cols = 4,
#     explode_world=False,
#     trim_world=False,
#     no_air = False,
#     alpha = False,
#     explode_alpha = True,
#     trim_alpha = True,
#     metric='euclidean',
# )

world states loop:   0%|          | 0/64 [00:00<?, ?it/s]

n cols 4, n rows 1, input dims (4, 64, 32, 32, 32)


3D plots:   0%|                                                                                               …

n cols 4, n rows 1, input dims (4, 64, 32, 32, 32)


3D plots:   0%|                                                                                               …

...

3D plots:   0%|                                                                                               …

n cols 4, n rows 1, input dims (4, 64, 32, 32, 32)


3D plots:   0%|                                                                                               …

n cols 4, n rows 1, input dims (4, 64, 32, 32, 32)


3D plots:   0%|                                                                                               …

## Metrics Visualise

In [ ]:
# load from tensorboard log